In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.6 MB/s eta 0:00:00
Mounted at /content/gdrive


In [2]:
from fastbook import *
from IPython.display import display, HTML

In [3]:
gdrive

Path('/content/gdrive/My Drive')

In [4]:
course_path = gdrive/'Colab Notebooks/fastbook/clean'
course_path

Path('/content/gdrive/My Drive/Colab Notebooks/fastbook/clean')

# NLP Deep Dive: RNNs

## Text Preprocessing

### Word Tokenization with fastai

In [5]:
from fastai.text.all import *

In [6]:
URLs.IMDB

'https://s3.amazonaws.com/fast-ai-nlp/imdb.tgz'

In [7]:
path = untar_data(URLs.IMDB)
path

Path('/root/.fastai/data/imdb')

In [8]:
files = get_text_files(path, folders=['train', 'test', 'unsup'])
files

(#100000) [Path('/root/.fastai/data/imdb/test/pos/3789_9.txt'),Path('/root/.fastai/data/imdb/test/pos/12336_7.txt'),Path('/root/.fastai/data/imdb/test/pos/12204_10.txt'),Path('/root/.fastai/data/imdb/test/pos/4867_10.txt'),Path('/root/.fastai/data/imdb/test/pos/5230_9.txt'),Path('/root/.fastai/data/imdb/test/pos/7799_10.txt'),Path('/root/.fastai/data/imdb/test/pos/5069_8.txt'),Path('/root/.fastai/data/imdb/test/pos/12327_10.txt'),Path('/root/.fastai/data/imdb/test/pos/6507_8.txt'),Path('/root/.fastai/data/imdb/test/pos/10807_7.txt')...]

In [9]:
with open(files[0], 'r') as f:
    txt = f.read()
txt[:75]

'Considering all of the comedies with a military situation that have been do'

---

Demo of `coll_repr`:

In [10]:
x = list(range(10))
x

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [11]:
coll_repr(x, 3)

'(#10) [0,1,2...]'

In [12]:
coll_repr(x, 5)

'(#10) [0,1,2,3,4...]'

In [13]:
type(coll_repr(x, 5))

str

In [14]:
print(coll_repr(x, 5))

(#10) [0,1,2,3,4...]


---

In [15]:
spacy = WordTokenizer()
type(spacy)

fastai.text.core.SpacyTokenizer

In [16]:
type(spacy([txt]))

generator

**Note:** The generator generates a list of tokens for each string. If we pass a list with more than one string, then we'll get back more than one list.

In [17]:
help(first)

Help on function first in module fastcore.basics:

first(x, f=None, negate=False, **kwargs)
    First element of `x`, optionally filtered by `f`, or None if missing



In [18]:
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#210) ['Considering','all','of','the','comedies','with','a','military','situation','that','have','been','done','in','history',',','someone','had','to','be','the','first','.','One','could','make','a','case','that','in'...]


In [19]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [20]:
tkn = Tokenizer(spacy)
type(tkn)

fastai.text.core.Tokenizer

In [21]:
type(tkn(txt))

fastcore.foundation.L

Notice two differences between `spacy` and `tkn`:

1. `spacy` accepts a collection of strings, whereas `tkn` accepts a single string.
2. `spacy` returns a generator, whereas `tkn` returns an `L` object.

In [22]:
print(coll_repr(tkn(txt), 31))

(#245) ['xxbos','xxmaj','considering','all','of','the','comedies','with','a','military','situation','that','have','been','done','in','history',',','someone','had','to','be','the','first','.','xxmaj','one','could','make','a','case'...]


In [23]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [24]:
??replace_rep

In [25]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

### Subword Tokenization

In [26]:
txts = []
for o in files[:2000]:
    with open(o, 'r') as f:
        txts.append(f.read())
txts = L(txts)
txts

(#2000) ["Considering all of the comedies with a military situation that have been done in history, someone had to be the first. One could make a case that in Shoulder Arms, Charlie Chaplin invented the genre.<br /><br />Hard to believe that back then this was a daring move. When you consider that some of the best films involving such people as Bob Hope, Abbott&Costello, Laurel&Hardy involved military service and made during war time, it's just something you accept and laugh at.<br /><br />In the First World War Chaplin along with fellow stars Douglas Fairbanks and Mary Pickford went out on bond tours. He was a great supporter of the Allied cause, unusual for someone of his left wing views. It would seem only natural that the Tramp would be drafted and unfortunately would flummox around and wreak havoc on all.<br /><br />A lot of things you'd see in the service comedies of World War II got their start in Shoulder Arms. Chaplin had no more imitators because within a few weeks of the fil

In [27]:
sp = SubwordTokenizer(vocab_sz=1000)
type(sp)

fastai.text.core.SentencePieceTokenizer

Recall that SentencePiece is a common choice is multi-lingual models (such as XLMRoBERTa).

In [28]:
sp.setup(txts)

{'sp_model': Path('tmp/spm.model')}

**Note:** We're passing in a list of strings to `setup`.

In [29]:
type(sp([txt]))

generator

In [30]:
toks = first(sp([txt]))
print(coll_repr(toks, 40))

(#410) ['▁Co','n','s','id','er','ing','▁all','▁of','▁the','▁come','d','ies','▁with','▁a','▁','mi','li','t','ar','y','▁situation','▁that','▁have','▁been','▁done','▁in','▁history',',','▁someone','▁had','▁to','▁be','▁the','▁first','.','▁O','ne','▁could','▁make','▁a'...]


In [31]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [32]:
subword(1000)

'▁Co n s id er ing ▁all ▁of ▁the ▁come d ies ▁with ▁a ▁ mi li t ar y ▁situation ▁that ▁have ▁been ▁done ▁in ▁history , ▁someone ▁had ▁to ▁be ▁the ▁first . ▁O ne ▁could ▁make ▁a'

In [33]:
subword(200)

'▁ C on s i d er ing ▁ al l ▁of ▁the ▁ com ed i es ▁with ▁a ▁ m i l it ar y ▁s it u a tion ▁that ▁have ▁be en ▁do n e ▁in'

In [34]:
subword(10000)

'▁Consider ing ▁all ▁of ▁the ▁comedies ▁with ▁a ▁military ▁situation ▁that ▁have ▁been ▁done ▁in ▁history , ▁someone ▁had ▁to ▁be ▁the ▁first . ▁One ▁could ▁make ▁a ▁case ▁that ▁in ▁Should er ▁Arm s , ▁Charlie ▁Chaplin ▁invent ed'

### Numericalization with fastai

In [35]:
toks = tkn(txt)
print(coll_repr(toks, 31))

(#245) ['xxbos','xxmaj','considering','all','of','the','comedies','with','a','military','situation','that','have','been','done','in','history',',','someone','had','to','be','the','first','.','xxmaj','one','could','make','a','case'...]


In [36]:
# Reminder:
type(txts), len(txts)

(fastcore.foundation.L, 2000)

In [37]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#245) ['xxbos','xxmaj','considering','all','of','the','comedies','with','a','military'...]

In [38]:
num = Numericalize()
type(num)

fastai.text.data.Numericalize

**Note:** When instantiating `Numericalize`, we can specify `min_freq` and `max_vocab`. The reason is that the spaCy tokenizer doesn't create a vocab with a fixed number of tokens. It also doesn't set a minimum frequency. But SentencePiece has a fixed vocabulary size!

In [39]:
num.setup(toks200)

**Note:** We're passing in a list of lists of tokens to `setup`.

In [40]:
type(num.vocab), len(num.vocab)

(list, 2024)

The length of the vocab is less because we only used 200 texts to set it up.

In [41]:
coll_repr(num.vocab, 20)

"(#2024) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','a','and','of','to','is','in','it','i'...]"

In [42]:
nums = num(toks)[:20]
nums

TensorText([   2,    8,  810,   47,   14,    9,  569,   31,   12, 1177,  570,   20,   43,  100,  151,   17,  513,   11,  288,   86])

In [43]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj considering all of the comedies with a military situation that have been done in history , someone had'

### Putting Our Texts into Batches for a Language Model

In [44]:
# Just run:
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In [45]:
# Just run:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


In [46]:
# Just run:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


In [47]:
# Just run:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


In [48]:
nums200 = toks200.map(num)
type(nums200)

fastcore.foundation.L

In [49]:
len(nums200)

200

In [50]:
nums200[0]

TensorText([   2,    8,  810,   47,   14,    9,  569,   31,   12, 1177,  570,   20,   43,  100,  151,   17,  513,   11,  288,   86,   15,   45,    9,  104,   10,    8,   39,  108,  126,   12,  514,
              20,   17,    8,    0,    8,    0,   11,    8,    0,    8,  571,    0,    9,  642,   10,   27,    8,  230,   15,  250,   20,  152,  117,   21,   24,   12,  811,  960,   10,    8,   72,
              33,  643,   20,   69,   14,    9,   99,  135,  711,  198,  111,   25,    8,    0,    8,  289,   11,    0,   11,    0,  515, 1177, 1178,   13,  116,  215,  316,   61,   11,   18,   22,
              73,  153,   33,  961,   13,  416,   50,   10,   27,    8,   17,    9,    8,  104,    8,  148,    8,  316,    8,  571,  337,   31,  812,  443,    8,    0,    8,    0,   13,    8,    0,
               8,    0,  572,   66,   35, 1470,    0,   10,    8,   36,   24,   12,   67,    0,   14,    9,    8,    0, 1471,   11, 1179,   26,  288,   14,   34,  444,    0, 1472,   10,    8,   18,
          

In [51]:
nums200[0].shape

torch.Size([245])

In [52]:
nums200[1]

TensorText([   2,   19,  101,   41,  149,  317,   57,   21,   28,  164,   19,  481,   18,   10,    8,   18,  299,   15,   45,   12,    0,   29,  417,  815,  362,   51,    9,  965,   22,   10,    8,
              59,    0,   87,   90,    9,  573,   18,    0,   51,   53,  114,   92,  124,   51,   20,  482,   10,   27,    8,    9,  104,  157,   19, 1474,   24,   94,  645,   21,   28,    0,   10,
               8,   18,   88,   41,   78,    9,  516,    0,   14,  517,  128,  129,    0,   39,   49,   39,   10,    8,   18,  418,  115,   61,    0,    9,  152,   79,   11,  816,   12,  154,  209,
              14, 1475,   57,   59,  318,  966,   10,   19,   24,  217,  817,   15,  105,   91,   39,   82,  109,   83,  103, 1476,   17,   21,   28,   10,    8,   30,  259,  818,   15,    9,    0,
             518,  215,    9,  231, 1477,  291,   11,   13,  117,   18,  176,  519, 1181,   38,   21,   24,   12,  171,   61,   15,  200,   12,    0,  714,  363,   19,  217,  108,   41,  967,   70,
          

In [53]:
nums200[1].shape

torch.Size([336])

In [54]:
dl = LMDataLoader(nums200)
type(dl)

fastai.text.data.LMDataLoader

In [55]:
x, y = first(dl)
x.shape, y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [56]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj considering all of the comedies with a military situation that have been done in history , someone had'

In [57]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj considering all of the comedies with a military situation that have been done in history , someone had to'

## Training a Text Classifier

### Language Model Using DataBlock

In [58]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])
dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb,
    splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [59]:
type(dls_lm)

fastai.data.core.DataLoaders

In [60]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj awesome xxunk classic from rko 's master breast man , xxmaj howard xxmaj hughes , this 3d ( cup ? ) extravaganza is a must see for anyone keen for desert showgirl antics and scantily clad harem chases … .every stripper and burlesque queen in the greater xxup la area must have got a gig on this notorious xxmaj technicolor romp . xxmaj in 3d it would have knocked xxup both your eyes out ( to paraphrase xxup","xxmaj awesome xxunk classic from rko 's master breast man , xxmaj howard xxmaj hughes , this 3d ( cup ? ) extravaganza is a must see for anyone keen for desert showgirl antics and scantily clad harem chases … .every stripper and burlesque queen in the greater xxup la area must have got a gig on this notorious xxmaj technicolor romp . xxmaj in 3d it would have knocked xxup both your eyes out ( to paraphrase xxup the"
1,"show in this film , and it is clear that at the time of making the film xxmaj truffaut was very much smitten with her . a definite must see for any cineaste or xxunk . 10 / 10 xxbos xxmaj gilles xxmaj xxunk is playing with the audience with the story of xxmaj max ( vincent xxmaj cassel ) in search of his biggest lost love ( monica xxmaj bellucci ) just before to get engaged to another woman","in this film , and it is clear that at the time of making the film xxmaj truffaut was very much smitten with her . a definite must see for any cineaste or xxunk . 10 / 10 xxbos xxmaj gilles xxmaj xxunk is playing with the audience with the story of xxmaj max ( vincent xxmaj cassel ) in search of his biggest lost love ( monica xxmaj bellucci ) just before to get engaged to another woman ."


Let's explore an actual minibatch comprising input IDs.

In [61]:
dls_lm[0]

This is the training data loader.

In [62]:
x_batch, y_batch = next(iter(dls_lm[0]))
x_batch.shape, y_batch.shape

(torch.Size([128, 80]), torch.Size([128, 80]))

In [63]:
x_batch[0]

LMTensorText([    2,     8,  1502,     8,  2448,    23,   180,    18,    20,    32,    16,    73,    15,    41,   679,  1233,    14,  7783,    11,    21,    16,    11,    40,  2290,     9,   279,
                264,    51,   272,    28,   934,    40,  5123,    17,    11,    12,    21,  1769,     9,  9712,     8,  1871,     8,  3094,    10,     8,     9,    30,    11,    82,    16,    70,
                 94,    13,   245, 17044,    11,   294,  1157,    83,    18,     9,   158,    69,     8,  2448,    16, 24532,    48,     9,  4846,    29,   177,  7364,    10,     8,    17,    16,
               3447,    11], device='cuda:0')

In [64]:
y_batch[0]

TensorText([    8,  1502,     8,  2448,    23,   180,    18,    20,    32,    16,    73,    15,    41,   679,  1233,    14,  7783,    11,    21,    16,    11,    40,  2290,     9,   279,   264,    51,
              272,    28,   934,    40,  5123,    17,    11,    12,    21,  1769,     9,  9712,     8,  1871,     8,  3094,    10,     8,     9,    30,    11,    82,    16,    70,    94,    13,   245,
            17044,    11,   294,  1157,    83,    18,     9,   158,    69,     8,  2448,    16, 24532,    48,     9,  4846,    29,   177,  7364,    10,     8,    17,    16,  3447,    11,    13],
           device='cuda:0')

In [65]:
type(dls_lm.vocab), len(dls_lm.vocab)

(list, 60008)

In [66]:
print(dls_lm.vocab[:30])

['xxunk', 'xxpad', 'xxbos', 'xxeos', 'xxfld', 'xxrep', 'xxwrep', 'xxup', 'xxmaj', 'the', '.', ',', 'and', 'a', 'of', 'to', 'is', 'it', 'in', 'i', 'this', 'that', '"', "'s", '-', 'was', '\n\n', 'as', 'with', 'for']


### Fine-Tuning the Language Model

In [67]:
learn = language_model_learner(dls_lm, AWD_LSTM, drop_mult=0.3, metrics=[accuracy, Perplexity()]).to_fp16()

In [68]:
type(learn)

fastai.text.learner.LMLearner

In [69]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.007018,3.900962,0.300772,49.449989,13:29


### Saving and Loading Models

In [70]:
learn.save('1epoch')

Path('/root/.fastai/data/imdb/models/1epoch.pth')

In [71]:
learn = learn.load('1epoch')

In [72]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.754637,3.762523,0.316355,43.056931,14:05
1,3.712708,3.703706,0.322964,40.597492,13:47
2,3.640662,3.656852,0.328514,38.739182,13:41
3,3.554577,3.620924,0.333046,37.372093,13:44
4,3.494730,3.597985,0.335874,36.524563,13:53
5,3.425341,3.581882,0.338499,35.941120,14:15
6,3.363039,3.572073,0.339733,35.590302,13:55
7,3.289480,3.569885,0.340553,35.512527,13:54
8,3.243838,3.572850,0.340971,35.617950,13:52
9,3.221302,3.577052,0.340762,35.767944,14:24


In [73]:
learn.save_encoder('finetuned')

### Text Generation

In [74]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)]

In [75]:
preds

["i liked this movie because I 've seen a couple of Abbott and Costello films and it was funny and i like how this movie was made . The Boys are also great and Abbott and Costello are",
 'i liked this movie because it was a good love story . It was a story of a married couple who decide to live in New York a very long time after their divorce . So much for their friends and']

### Creating the Classifier DataLoaders

In [ ]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [ ]:
dls_clas.show_batch(max_n=3)

In [ ]:
nums_samp = toks200[:10].map(num)

In [ ]:
nums_samp.map(len)

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5,
                                metrics=accuracy).to_fp16()

In [ ]:
learn = learn.load_encoder('finetuned')

### Fine-Tuning the Classifier

In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

## Disinformation and Language Models

## Conclusion

## Questionnaire

1. What is "self-supervised learning"?
1. What is a "language model"?
1. Why is a language model considered self-supervised?
1. What are self-supervised models usually used for?
1. Why do we fine-tune language models?
1. What are the three steps to create a state-of-the-art text classifier?
1. How do the 50,000 unlabeled movie reviews help us create a better text classifier for the IMDb dataset?
1. What are the three steps to prepare your data for a language model?
1. What is "tokenization"? Why do we need it?
1. Name three different approaches to tokenization.
1. What is `xxbos`?
1. List four rules that fastai applies to text during tokenization.
1. Why are repeated characters replaced with a token showing the number of repetitions and the character that's repeated?
1. What is "numericalization"?
1. Why might there be words that are replaced with the "unknown word" token?
1. With a batch size of 64, the first row of the tensor representing the first batch contains the first 64 tokens for the dataset. What does the second row of that tensor contain? What does the first row of the second batch contain? (Careful—students often get this one wrong! Be sure to check your answer on the book's website.)
1. Why do we need padding for text classification? Why don't we need it for language modeling?
1. What does an embedding matrix for NLP contain? What is its shape?
1. What is "perplexity"?
1. Why do we have to pass the vocabulary of the language model to the classifier data block?
1. What is "gradual unfreezing"?
1. Why is text generation always likely to be ahead of automatic identification of machine-generated texts?

### Further Research

1. See what you can learn about language models and disinformation. What are the best language models today? Take a look at some of their outputs. Do you find them convincing? How could a bad actor best use such a model to create conflict and uncertainty?
1. Given the limitation that models are unlikely to be able to consistently recognize machine-generated texts, what other approaches may be needed to handle large-scale disinformation campaigns that leverage deep learning?